In [3]:
import numpy as np
import pandas as pd
import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.models.phrases import Phrases, Phraser
from gensim import corpora
from pathlib import Path
from sklearn.decomposition import PCA
import umap

In [13]:
from sklearn.preprocessing import LabelEncoder

In [4]:
data_dir = Path().cwd().parent / "data"

In [5]:
model0 = Doc2Vec.load("../models/all_model")

In [34]:
speaker_tags = model0.dv.index_to_key

speaker_tags = [tag for tag in speaker_tags if ("_D_" in tag) or ("_R_" in tag)]

embeds = np.array([model0.dv[tag] for tag in speaker_tags])

In [39]:
party = np.array([i.split("_")[1] for i in speaker_tags])

enc = LabelEncoder()
party = enc.fit_transform(party)

In [68]:
pca = umap.UMAP(n_components =1, n_neighbors=20).fit_transform(embeds)


In [69]:
pca_df = pd.DataFrame(pca, columns=["pc1"])

In [70]:
pca_df["tag"] = speaker_tags

pca_df[["unique_id","party","congress"]] = pca_df["tag"].str.split('_', n=2, expand=True)
pca_df["speakerid"] = pca_df["congress"] + pca_df["unique_id"] 

def make_id(x): 
    if len(x)==8: 
        x=x+"0"
    if len(x)==7: 
        x=[*x]
        x.insert(2, "1")
        x="".join(x)
    return str(x)

pca_df["speakerid"] = pca_df["speakerid"].map(make_id)

In [71]:
pca_df

,pc1,tag,unique_id,party,congress,speakerid
0,3.208749,10610_R_97,10610,R,97,97110610
1,4.546584,07450_D_97,07450,D,97,97107450
2,2.017025,10890_D_97,10890,D,97,97110890
3,4.407548,07370_D_97,07370,D,97,97107370
4,13.631512,07830_R_97,07830,R,97,97107830
...,...,...,...,...,...,...
7898,5.786085,12262_D_114,12262,D,114,114122620
7899,5.692798,12346_R_114,12346,R,114,114123460
7900,6.121677,11921_R_114,11921,R,114,114119210
7901,8.998811,11992_D_114,11992,D,114,114119920


In [72]:
speakers = pd.read_csv(data_dir / "speakers_all.csv")
speakers["speakerid"] = speakers["speakerid"].map(str)

In [73]:
pca_df = pca_df.merge(speakers, how="left", on="speakerid")

In [74]:
pca_df

,pc1,tag,unique_id,party,congress,speakerid,lastname,firstname,state,district
0,3.208749,10610_R_97,10610,R,97,97110610,MICHEL,ROBERT,IL,18.0
1,4.546584,07450_D_97,07450,D,97,97107450,ONEILL,THOMAS,MA,8.0
2,2.017025,10890_D_97,10890,D,97,97110890,WHITTEN,JAMIE,MS,1.0
3,4.407548,07370_D_97,07370,D,97,97107370,LONG,GILLIS,LA,8.0
4,13.631512,07830_R_97,07830,R,97,97107830,KEMP,JACK,NY,38.0
...,...,...,...,...,...,...,...,...,...,...
7898,5.786085,12262_D_114,12262,D,114,114122620,VELA,FILEMON,TX,34.0
7899,5.692798,12346_R_114,12346,R,114,114123460,LAHOOD,DARIN,IL,18.0
7900,6.121677,11921_R_114,11921,R,114,114119210,DESJARLAIS,SCOTT,TN,4.0
7901,8.998811,11992_D_114,11992,D,114,114119920,JACKSON LEE,SHEILA,TX,18.0


In [75]:
pca_df.to_csv(data_dir / "umap_all.csv", index=False)